<a href="https://colab.research.google.com/github/Harshanand7/NLP/blob/main/bert_model_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -U

In [2]:
!pip install datasets

In [3]:
import pandas as pd

In [4]:
from datasets import load_dataset

dataset = load_dataset("christophsonntag/OLID")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
df = dataset["train"].to_pandas()
df=df[['cleaned_tweet','subtask_a']]
df.head()

,cleaned_tweet,subtask_a
0,ask native americans take,OFF
1,go home drunk maga trump 2020,OFF
2,amazon investigating chinese employees selling...,NOT
3,someone vetaken piece shit volcano,OFF
4,obama wanted liberals illegals move red states,NOT


In [6]:


# Map 'OFF' to 1 and 'NOT' to 0 in the 'subtask_a' column
df['subtask_a'] = df['subtask_a'].map({'OFF': 1, 'NOT': 0})

# Display the first few rows to confirm the changes
print(df.head())

                                       cleaned_tweet  subtask_a
0                          ask native americans take          1
1                      go home drunk maga trump 2020          1
2  amazon investigating chinese employees selling...          0
3                 someone vetaken piece shit volcano          1
4     obama wanted liberals illegals move red states          0


In [7]:
output_file = 'modified_tweets.csv'
df.to_csv(output_file, index=False)

In [8]:

data = pd.read_csv("/content/modified_tweets.csv",  engine="python")
data.head()
data=data[['cleaned_tweet','subtask_a']]

In [9]:

data = data[0:13000]
data.head()

,cleaned_tweet,subtask_a
0,ask native americans take,1
1,go home drunk maga trump 2020,1
2,amazon investigating chinese employees selling...,0
3,someone vetaken piece shit volcano,1
4,obama wanted liberals illegals move red states,0


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
model = model.to('cuda')

In [14]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [15]:
X = list(data["cleaned_tweet"])
y = list(data["subtask_a"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(
    [str(text) for text in X_train],  # Convert each element in X_train to a string
    padding=True,
    truncation=True,
    max_length=512
)
X_val_tokenized = tokenizer(
    [str(text) for text in X_val],  # Convert each element in X_val to a string
    padding=True,
    truncation=True,
    max_length=512
)

In [16]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [17]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
len(X_train),len(X_val)

(10400, 2600)

In [19]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [20]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [21]:
train_dataset[5]

{'input_ids': tensor([  101, 22091,  2860,  2232,  4067,  2632,  9541,  4140,  1038, 25459,
          2072,  4086,  2293,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [22]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [23]:
!pip install accelerate -U

In [24]:
# Define Trainer
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    logging_steps=20,  # Log every 10 steps
    report_to="none"  # Disable other reporting to focus on logs


)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

Step,Training Loss
20,0.654100
40,0.597200
60,0.622800
80,0.623300
100,0.570700
120,0.610200
140,0.507100
160,0.553400
180,0.557400
200,0.498200


TrainOutput(global_step=2600, training_loss=0.44054177669378425, metrics={'train_runtime': 408.5506, 'train_samples_per_second': 50.912, 'train_steps_per_second': 6.364, 'total_flos': 737533177056000.0, 'train_loss': 0.44054177669378425, 'epoch': 2.0})

In [26]:
trainer.evaluate()
#0.782

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.5185332298278809,
 'eval_accuracy': 0.7873076923076923,
 'eval_precision': 0.6899509803921569,
 'eval_recall': 0.6523754345307068,
 'eval_f1': 0.6706372840976772,
 'eval_runtime': 16.0294,
 'eval_samples_per_second': 162.201,
 'eval_steps_per_second': 20.275,
 'epoch': 2.0}

In [27]:
np.set_printoptions(suppress=True)

In [39]:
from sklearn.metrics import f1_score

predictions = []

# Ensure 'cleaned_tweet' column contains strings and handle missing values
for text in data['cleaned_tweet'].astype(str).fillna(''):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        probabilities = probabilities.cpu().detach().numpy()

    non_offensive_prob = probabilities[0][0]  # Probability of being non-offensive
    if non_offensive_prob >= 0.67:
        predictions.append(0)  # Non-offensive
    else:
        predictions.append(1)  # Offensive

data['predictions'] = predictions

# Calculate the F1 score
f1 = f1_score(data['subtask_a'], data['predictions'])  # Ensure 'subtask_a' is your true label column
print(f"F1 Score: {f1}")

# Save the dataset with predictions
data.to_csv('predictions.csv', index=False)

# Optional: Display the first few rows of the dataframe
print(data.head())

F1 Score: 0.8281214848143981
                                       cleaned_tweet  subtask_a  predictions
0                          ask native americans take          1            0
1                      go home drunk maga trump 2020          1            1
2  amazon investigating chinese employees selling...          0            0
3                 someone vetaken piece shit volcano          1            1
4     obama wanted liberals illegals move red states          0            0


#the next code blocks are only for testing


In [28]:
text = "That was a good point"

inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions
non_offensive_prob_1 = predictions[0][0]
#The output is an array of two elements where the first element is the probability of the test to be not offensive
#and the second one is the probability of text as  offensive

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.1267, -1.5264]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9747, 0.0253]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [29]:
if (non_offensive_prob_1>=0.67):
  print("The given text is likely to be classified as NON Offensive text")
else:
  print("The given text is probably OFFENSIVE text")

The given text is likely to be classified as NON Offensive text


In [30]:
trainer.save_model('CustomModel')

In [31]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [32]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [33]:

text = "you are a fool"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions
non_offensive_prob = predictions[0][0]

#The output is an array of two elements where the first element is the probability of the test to be not offensive
#and the second one is the probability of text as  offensive



In [34]:
if (non_offensive_prob>=0.67):
  print("The given text is likely to be classified as NON Offensive text")
else:
  print("The given text is probably OFFENSIVE text")

The given text is probably OFFENSIVE text
